In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
from pathlib import Path

base_dir = "/content/drive/MyDrive/ASVspoof2021"
train_flac_dir = os.path.join(base_dir, "ASVspoof2021_DF_train", "flac")
dev_flac_dir = os.path.join(base_dir, "ASVspoof2021_DF_dev", "flac")
eval_flac_dir = os.path.join(base_dir, "ASVspoof2021_DF_eval", "flac")


Mounted at /content/drive


In [2]:
import os

base_path = "/content/drive/MyDrive/ASVspoof2021"


for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".tar.gz") or file.endswith(".zip"):
            print("Found archive:", os.path.join(root, file))



Found archive: /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval_part00.tar.gz
Found archive: /content/drive/MyDrive/ASVspoof2021/keys/PA-keys-full.tar.gz
Found archive: /content/drive/MyDrive/ASVspoof2021/keys/DF-keys-full.tar.gz
Found archive: /content/drive/MyDrive/ASVspoof2021/keys/LA-keys-full.tar.gz
Found archive: /content/drive/MyDrive/ASVspoof2021/DF_download/ASVspoof2021_DF_eval_part00.tar.gz
Found archive: /content/drive/MyDrive/ASVspoof2021/DF_download/ASVspoof2021_DF_eval_part01.tar.gz
Found archive: /content/drive/MyDrive/ASVspoof2021/DF_download/ASVspoof2021_DF_eval_part02.tar.gz


In [3]:
import os

meta_path = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt"
print("Exists:", os.path.exists(meta_path))


Exists: True


In [4]:

input_dir = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"
output_dir = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/wav_for_rawnet"


In [6]:
!git clone https://github.com/Jungjee/RawNet.git
%cd RawNet



Cloning into 'RawNet'...
remote: Enumerating objects: 619, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 619 (delta 38), reused 45 (delta 25), pack-reused 536 (from 1)
Receiving objects: 100% (619/619), 180.00 MiB | 30.02 MiB/s, done.
Resolving deltas: 100% (319/319), done.
Updating files: 100% (65/65), done.
/content/RawNet


In [5]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


GPU Available: False
Device Name: CPU


In [6]:
import os
import urllib.request
import tarfile

BASE_URL = "https://zenodo.org/records/14498691/files/"
DATASET_PATH = "/content/"
protocol_filename = "ASVspoof5_protocols.tar.gz"
protocol_filepath = os.path.join(DATASET_PATH, protocol_filename)


if not os.path.exists(protocol_filepath):
    print(f"Downloading {protocol_filename}...")
    urllib.request.urlretrieve(BASE_URL + protocol_filename, protocol_filepath)
    print("Download complete.")
else:
    print(f"{protocol_filename} already exists.")

extracted_folder = os.path.join(DATASET_PATH, "ASVspoof5_protocols")
if not os.path.exists(extracted_folder):
    print("Extracting files...")
    with tarfile.open(protocol_filepath, "r:gz") as tar:
        tar.extractall(DATASET_PATH)
    print("Extraction complete.")
else:
    print("Files already extracted.")


Download complete.
Extracting files...
Extraction complete.


In [9]:
!tar -xvzf /content/ASVspoof5_protocols.tar.gz -C /content/


ASVspoof5.codec.config.csv
ASVspoof5.dev.track_1.tsv
ASVspoof5.dev.track_2.enroll.tsv
ASVspoof5.dev.track_2.trial.tsv
ASVspoof5.eval.track_1.tsv
ASVspoof5.eval.track_2.enroll.tsv
ASVspoof5.eval.track_2.trial.tsv
ASVspoof5.train.tsv


In [7]:
import pandas as pd

protocol_file = "/content/ASVspoof5.train.tsv"
df_protocol = pd.read_csv(protocol_file, sep="\t")

print(df_protocol.head())




   T_4850 T_0000000000 F - - - AC3 A05 spoof -
0  T_0858 T_0000000001 M - - - AC3 A07 spoof -
1  T_4075 T_0000000002 M - - - AC2 A04 spoof -
2  T_0938 T_0000000003 M - - - AC2 A08 spoof -
3  T_0610 T_0000000004 M - - - AC2 A05 spoof -
4  T_2896 T_0000000005 F - - - AC2 A05 spoof -


In [8]:
df_protocol = pd.read_csv(protocol_file, sep=" ", header=None)
print(df_protocol.head())
print(df_protocol.shape)


        0             1  2  3  4  5    6    7      8  9
0  T_4850  T_0000000000  F  -  -  -  AC3  A05  spoof  -
1  T_0858  T_0000000001  M  -  -  -  AC3  A07  spoof  -
2  T_4075  T_0000000002  M  -  -  -  AC2  A04  spoof  -
3  T_0938  T_0000000003  M  -  -  -  AC2  A08  spoof  -
4  T_0610  T_0000000004  M  -  -  -  AC2  A05  spoof  -
(182357, 10)


In [9]:
import pandas as pd


columns = ["speaker_id", "file_name", "gender", "col1", "col2", "col3", "codec", "attack_id", "label", "col4"]

df_protocol = pd.read_csv("/content/ASVspoof5.train.tsv", sep=" ", header=None, names=columns)


df_protocol = df_protocol.drop(columns=["col1", "col2", "col3", "col4"])


df_protocol["label"] = df_protocol["label"].map({"bonafide": 1, "spoof": 0})


print(df_protocol.head())
print(df_protocol.info())


  speaker_id     file_name gender codec attack_id  label
0     T_4850  T_0000000000      F   AC3       A05      0
1     T_0858  T_0000000001      M   AC3       A07      0
2     T_4075  T_0000000002      M   AC2       A04      0
3     T_0938  T_0000000003      M   AC2       A08      0
4     T_0610  T_0000000004      M   AC2       A05      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182357 entries, 0 to 182356
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   speaker_id  182357 non-null  object
 1   file_name   182357 non-null  object
 2   gender      182357 non-null  object
 3   codec       182357 non-null  object
 4   attack_id   182357 non-null  object
 5   label       182357 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 8.3+ MB
None


In [16]:
!tar -xvzf /content/ASVspoof5_protocols.tar.gz -C /content/


ASVspoof5.codec.config.csv
ASVspoof5.dev.track_1.tsv
ASVspoof5.dev.track_2.enroll.tsv
ASVspoof5.dev.track_2.trial.tsv
ASVspoof5.eval.track_1.tsv
ASVspoof5.eval.track_2.enroll.tsv
ASVspoof5.eval.track_2.trial.tsv
ASVspoof5.train.tsv


In [14]:
import pandas as pd

protocol_file = "/content/ASVspoof5.train.tsv"
df = pd.read_csv(protocol_file, sep="\s+", header=None)


print(df.head())
print(df.shape)


        0             1  2  3  4  5    6    7      8  9
0  T_4850  T_0000000000  F  -  -  -  AC3  A05  spoof  -
1  T_0858  T_0000000001  M  -  -  -  AC3  A07  spoof  -
2  T_4075  T_0000000002  M  -  -  -  AC2  A04  spoof  -
3  T_0938  T_0000000003  M  -  -  -  AC2  A08  spoof  -
4  T_0610  T_0000000004  M  -  -  -  AC2  A05  spoof  -
(182357, 10)


In [15]:

df.columns = ["speaker_id", "file_name", "gender", "drop1", "drop2", "drop3", "codec", "attack_id", "label", "drop4"]


df = df.drop(columns=["drop1", "drop2", "drop3", "drop4"])


df["label"] = df["label"].map({"spoof": 0, "bonafide": 1})


print(df.head())
print(df.info())



  speaker_id     file_name gender codec attack_id  label
0     T_4850  T_0000000000      F   AC3       A05      0
1     T_0858  T_0000000001      M   AC3       A07      0
2     T_4075  T_0000000002      M   AC2       A04      0
3     T_0938  T_0000000003      M   AC2       A08      0
4     T_0610  T_0000000004      M   AC2       A05      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182357 entries, 0 to 182356
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   speaker_id  182357 non-null  object
 1   file_name   182357 non-null  object
 2   gender      182357 non-null  object
 3   codec       182357 non-null  object
 4   attack_id   182357 non-null  object
 5   label       182357 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 8.3+ MB
None


In [16]:
!wget --content-disposition "https://zenodo.org/api/records/4835108/files/ASVspoof2021_DF_eval_part00.tar.gz/content" -P /content/
!wget --content-disposition "https://zenodo.org/api/records/4835108/files/ASVspoof2021_DF_eval_part01.tar.gz/content" -P /content/
!wget --content-disposition "https://zenodo.org/api/records/4835108/files/ASVspoof2021_DF_eval_part02.tar.gz/content" -P /content/
!wget --content-disposition "https://zenodo.org/api/records/4835108/files/ASVspoof2021_DF_eval_part03.tar.gz/content" -P /content/


--2025-04-06 15:38:09--  https://zenodo.org/api/records/4835108/files/ASVspoof2021_DF_eval_part00.tar.gz/content
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.43.25, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8637050238 (8.0G) [application/octet-stream]
Saving to: ‘/content/ASVspoof2021_DF_eval_part00.tar.gz’

                      0%[                    ]  20.44M   968KB/s    eta 2h 20m ^C
--2025-04-06 15:38:31--  https://zenodo.org/api/records/4835108/files/ASVspoof2021_DF_eval_part01.tar.gz/content
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.43.25, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8624944890 (8.0G) [application/octet-stream]
Saving to: ‘/content/ASVspoof2021_DF_eval_part01.tar.gz’

021_DF_eval_part01.   0%[                    ]   3.26M   8

In [58]:
!apt-get install aria2 -y
!aria2c -x 16 -s 16 "https://zenodo.org/api/records/4835108/files/ASVspoof2021_DF_eval_part00.tar.gz/content" -d /content/
!aria2c -x 16 -s 16 "https://zenodo.org/api/records/4835108/files/ASVspoof2021_DF_eval_part01.tar.gz/content" -d /content/
!aria2c -x 16 -s 16 "https://zenodo.org/api/records/4835108/files/ASVspoof2021_DF_eval_part02.tar.gz/content" -d /content/
!aria2c -x 16 -s 16 "https://zenodo.org/api/records/4835108/files/ASVspoof2021_DF_eval_part03.tar.gz/content" -d /content/


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.3 [45.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1,086 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1,513 kB in 1s (1,020 kB/s)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 126210 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubun

In [12]:
mkdir -p /content/drive/MyDrive/ASVspoof2021


In [1]:
!mv /content/ASVspoof2021_DF_eval_part*.tar.gz /content/drive/MyDrive/ASVspoof2021/


In [7]:
!tar -xvzf /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval_part00.tar.gz -C /content/drive/MyDrive/ASVspoof2021/
!tar -xvzf /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval_part01.tar.gz -C /content/drive/MyDrive/ASVspoof2021/
!tar -xvzf /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval_part02.tar.gz -C /content/drive/MyDrive/ASVspoof2021/
!tar -xvzf /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval_part03.tar.gz -C /content/drive/MyDrive/ASVspoof2021/


ASVspoof2021_DF_eval/ASVspoof2021.DF.cm.eval.trl.txt
ASVspoof2021_DF_eval/LICENSE.DF.txt
ASVspoof2021_DF_eval/README.DF.txt
ASVspoof2021_DF_eval/flac/DF_E_2000011.flac
ASVspoof2021_DF_eval/flac/DF_E_2000013.flac
ASVspoof2021_DF_eval/flac/DF_E_2000024.flac
ASVspoof2021_DF_eval/flac/DF_E_2000026.flac
ASVspoof2021_DF_eval/flac/DF_E_2000027.flac
ASVspoof2021_DF_eval/flac/DF_E_2000028.flac
ASVspoof2021_DF_eval/flac/DF_E_2000031.flac
ASVspoof2021_DF_eval/flac/DF_E_2000032.flac
ASVspoof2021_DF_eval/flac/DF_E_2000040.flac
ASVspoof2021_DF_eval/flac/DF_E_2000042.flac
ASVspoof2021_DF_eval/flac/DF_E_2000044.flac
ASVspoof2021_DF_eval/flac/DF_E_2000048.flac
ASVspoof2021_DF_eval/flac/DF_E_2000049.flac
ASVspoof2021_DF_eval/flac/DF_E_2000053.flac
ASVspoof2021_DF_eval/flac/DF_E_2000055.flac
ASVspoof2021_DF_eval/flac/DF_E_2000058.flac
ASVspoof2021_DF_eval/flac/DF_E_2000072.flac
ASVspoof2021_DF_eval/flac/DF_E_2000075.flac
ASVspoof2021_DF_eval/flac/DF_E_2000079.flac
ASVspoof2021_DF_eval/flac/DF_E_2000080.f

In [2]:
data_path = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"


In [3]:
import os

data_path = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"

if os.path.exists(data_path):
    files = os.listdir(data_path)
    print(f"Total audio files: {len(files)}")
    print("Sample files:", files[:5])  # Show first 5 files
else:
    print("❌ Directory not found! Check the path.")



Total audio files: 2231
Sample files: ['DF_E_2000829.flac', 'DF_E_2753491.flac', 'DF_E_2000946.flac', 'DF_E_2751336.flac', 'DF_E_2753263.flac']


In [10]:
data_path = "/content/drive/MyDrive/ASVspoof2021/flac"



In [11]:
import os

data_path = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"
files = os.listdir(data_path)

print(f"Total audio files: {len(files)}")
print("Sample files:", files[:5])




Total audio files: 2232
Sample files: ['DF_E_2753491.flac', 'DF_E_2000946.flac', 'DF_E_2751336.flac', 'DF_E_2753263.flac', 'DF_E_2001196.flac']


In [12]:
import random
import IPython.display as ipd


sample_file = random.choice(files)
sample_path = os.path.join(data_path, sample_file)

# Play the audio
print(f"Playing sample: {sample_file}")
ipd.Audio(sample_path)


Playing sample: DF_E_2750545.flac


In [8]:
data_path = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"



In [22]:
DATA_DIR = "/content/drive/MyDrive/ASVspoof2021/keys"
!mkdir -p {DATA_DIR}


In [23]:
!DATA_DIR="/content/drive/MyDrive/ASVspoof2021/keys"
!mkdir -p $DATA_DIR

!echo -e "https://www.asvspoof.org/asvspoof2021/LA-keys-full.tar.gz\nhttps://www.asvspoof.org/asvspoof2021/PA-keys-full.tar.gz\nhttps://www.asvspoof.org/asvspoof2021/DF-keys-full.tar.gz" | xargs -n 1 -P 3 wget -P $DATA_DIR



--2025-04-05 12:42:38--  https://www.asvspoof.org/asvspoof2021/DF-keys-full.tar.gz
--2025-04-05 12:42:38--  https://www.asvspoof.org/asvspoof2021/LA-keys-full.tar.gz
Resolving www.asvspoof.org (www.asvspoof.org)... --2025-04-05 12:42:38--  https://www.asvspoof.org/asvspoof2021/PA-keys-full.tar.gz
Resolving www.asvspoof.org (www.asvspoof.org)... Resolving www.asvspoof.org (www.asvspoof.org)... 193.55.113.222
Connecting to www.asvspoof.org (www.asvspoof.org)|193.55.113.222|:443... 193.55.113.222
Connecting to www.asvspoof.org (www.asvspoof.org)|193.55.113.222|:443... 193.55.113.222
Connecting to www.asvspoof.org (www.asvspoof.org)|193.55.113.222|:443... connected.
connected.
connected.
HTTP request sent, awaiting response... HTTP request sent, awaiting response... HTTP request sent, awaiting response... ^C


In [21]:
!tar -xvzf /content/drive/MyDrive/ASVspoof2021/keys/DF-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys
!tar -xvzf /content/drive/MyDrive/ASVspoof2021/keys/LA-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys
!tar -xvzf /content/drive/MyDrive/ASVspoof2021/keys/PA-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys


keys/DF/
keys/DF/CM/
keys/DF/CM/trial_metadata.txt
keys/DF/CM/LFCC-GMM/
keys/DF/CM/LFCC-GMM/score.txt
keys/DF/CM/LFCC-LCNN/
keys/DF/CM/LFCC-LCNN/score.txt
keys/DF/CM/RawNet2/
keys/DF/CM/RawNet2/score.txt
keys/DF/CM/CQCC-GMM/
keys/DF/CM/CQCC-GMM/score.txt
keys/DF/README.txt
keys/LA/
keys/LA/README.txt
keys/LA/ASV/
keys/LA/ASV/ASVTorch_Kaldi/
keys/LA/ASV/ASVTorch_Kaldi/score.txt
keys/LA/ASV/trial_metadata.txt
keys/LA/CM/
keys/LA/CM/trial_metadata.txt
keys/LA/CM/LFCC-GMM/
keys/LA/CM/LFCC-GMM/score.txt
keys/LA/CM/LFCC-LCNN/
keys/LA/CM/LFCC-LCNN/score.txt
keys/LA/CM/RawNet2/
keys/LA/CM/RawNet2/score.txt
keys/LA/CM/CQCC-GMM/
keys/LA/CM/CQCC-GMM/score.txt
keys/LA/LA-C012-eval.npy
keys/LA/LA-C012-prog.npy
keys/LA/LA-C012-hidden.npy
keys/PA/
keys/PA/ASV/
keys/PA/ASV/trial_metadata.txt
keys/PA/ASV/ASVTorch_Kaldi/
keys/PA/ASV/ASVTorch_Kaldi/score.txt
keys/PA/CM/
keys/PA/CM/trial_metadata.txt
keys/PA/CM/LFCC-GMM/
keys/PA/CM/LFCC-GMM/score.txt
keys/PA/CM/LFCC-LCNN/
keys/PA/CM/LFCC-LCNN/score.txt
ke

In [22]:
!tar -xvf /content/drive/MyDrive/ASVspoof2021/keys/DF-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys/
!tar -xvf /content/drive/MyDrive/ASVspoof2021/keys/LA-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys/
!tar -xvf /content/drive/MyDrive/ASVspoof2021/keys/PA-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys/


keys/DF/
keys/DF/CM/
keys/DF/CM/trial_metadata.txt
keys/DF/CM/LFCC-GMM/
keys/DF/CM/LFCC-GMM/score.txt
keys/DF/CM/LFCC-LCNN/
keys/DF/CM/LFCC-LCNN/score.txt
keys/DF/CM/RawNet2/
keys/DF/CM/RawNet2/score.txt
keys/DF/CM/CQCC-GMM/
keys/DF/CM/CQCC-GMM/score.txt
keys/DF/README.txt
keys/LA/
keys/LA/README.txt
keys/LA/ASV/
keys/LA/ASV/ASVTorch_Kaldi/
keys/LA/ASV/ASVTorch_Kaldi/score.txt
keys/LA/ASV/trial_metadata.txt
keys/LA/CM/
keys/LA/CM/trial_metadata.txt
keys/LA/CM/LFCC-GMM/
keys/LA/CM/LFCC-GMM/score.txt
keys/LA/CM/LFCC-LCNN/
keys/LA/CM/LFCC-LCNN/score.txt
keys/LA/CM/RawNet2/
keys/LA/CM/RawNet2/score.txt
keys/LA/CM/CQCC-GMM/
keys/LA/CM/CQCC-GMM/score.txt
keys/LA/LA-C012-eval.npy
keys/LA/LA-C012-prog.npy
keys/LA/LA-C012-hidden.npy
keys/PA/
keys/PA/ASV/
keys/PA/ASV/trial_metadata.txt
keys/PA/ASV/ASVTorch_Kaldi/
keys/PA/ASV/ASVTorch_Kaldi/score.txt
keys/PA/CM/
keys/PA/CM/trial_metadata.txt
keys/PA/CM/LFCC-GMM/
keys/PA/CM/LFCC-GMM/score.txt
keys/PA/CM/LFCC-LCNN/
keys/PA/CM/LFCC-LCNN/score.txt
ke

In [23]:
!tar -xvf /content/drive/MyDrive/ASVspoof2021/keys/DF-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys/
!tar -xvf /content/drive/MyDrive/ASVspoof2021/keys/LA-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys/
!tar -xvf /content/drive/MyDrive/ASVspoof2021/keys/PA-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys/


keys/DF/
keys/DF/CM/
keys/DF/CM/trial_metadata.txt
keys/DF/CM/LFCC-GMM/
keys/DF/CM/LFCC-GMM/score.txt
keys/DF/CM/LFCC-LCNN/
keys/DF/CM/LFCC-LCNN/score.txt
keys/DF/CM/RawNet2/
keys/DF/CM/RawNet2/score.txt
keys/DF/CM/CQCC-GMM/
keys/DF/CM/CQCC-GMM/score.txt
keys/DF/README.txt
keys/LA/
keys/LA/README.txt
keys/LA/ASV/
keys/LA/ASV/ASVTorch_Kaldi/
keys/LA/ASV/ASVTorch_Kaldi/score.txt
keys/LA/ASV/trial_metadata.txt
keys/LA/CM/
keys/LA/CM/trial_metadata.txt
keys/LA/CM/LFCC-GMM/
keys/LA/CM/LFCC-GMM/score.txt
keys/LA/CM/LFCC-LCNN/
keys/LA/CM/LFCC-LCNN/score.txt
keys/LA/CM/RawNet2/
keys/LA/CM/RawNet2/score.txt
keys/LA/CM/CQCC-GMM/
keys/LA/CM/CQCC-GMM/score.txt
keys/LA/LA-C012-eval.npy
keys/LA/LA-C012-prog.npy
keys/LA/LA-C012-hidden.npy
keys/PA/
keys/PA/ASV/
keys/PA/ASV/trial_metadata.txt
keys/PA/ASV/ASVTorch_Kaldi/
keys/PA/ASV/ASVTorch_Kaldi/score.txt
keys/PA/CM/
keys/PA/CM/trial_metadata.txt
keys/PA/CM/LFCC-GMM/
keys/PA/CM/LFCC-GMM/score.txt
keys/PA/CM/LFCC-LCNN/
keys/PA/CM/LFCC-LCNN/score.txt
ke

In [24]:
!rm -rf /content/drive/MyDrive/ASVspoof2021/keys/DF/
!rm -rf /content/drive/MyDrive/ASVspoof2021/keys/LA/
!rm -rf /content/drive/MyDrive/ASVspoof2021/keys/PA/



In [20]:
!tar -xvf /content/drive/MyDrive/ASVspoof2021/keys/DF-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys/
!ls -lh /content/drive/MyDrive/ASVspoof2021/keys/


keys/DF/
keys/DF/CM/
keys/DF/CM/trial_metadata.txt
keys/DF/CM/LFCC-GMM/
keys/DF/CM/LFCC-GMM/score.txt
keys/DF/CM/LFCC-LCNN/
keys/DF/CM/LFCC-LCNN/score.txt
keys/DF/CM/RawNet2/
keys/DF/CM/RawNet2/score.txt
keys/DF/CM/CQCC-GMM/
keys/DF/CM/CQCC-GMM/score.txt
keys/DF/README.txt
total 235M
drwx------ 3 root root 4.0K Oct  1  2022 DF
-rw------- 1 root root  25M Nov 28 13:36 DF-keys-full.tar.gz
-rw------- 1 root root  25M Nov 28 13:36 DF-keys-full.tar.gz.1
drwx------ 3 root root 4.0K Apr  4 13:03 keys
drwx------ 4 root root 4.0K Apr 13  2023 LA
-rw------- 1 root root  21M Nov 28 13:36 LA-keys-full.tar.gz
-rw------- 1 root root  21M Nov 28 13:36 LA-keys-full.tar.gz.1
drwx------ 4 root root 4.0K Oct  1  2022 PA
-rw------- 1 root root  73M Nov 28 13:36 PA-keys-full.tar.gz
-rw------- 1 root root  73M Nov 28 13:36 PA-keys-full.tar.gz.1


In [22]:
!tar -xvf /content/drive/MyDrive/ASVspoof2021/keys/DF-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys/
!tar -xvf /content/drive/MyDrive/ASVspoof2021/keys/LA-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys/
!tar -xvf /content/drive/MyDrive/ASVspoof2021/keys/PA-keys-full.tar.gz -C /content/drive/MyDrive/ASVspoof2021/keys/



keys/DF/
keys/DF/CM/
keys/DF/CM/trial_metadata.txt
keys/DF/CM/LFCC-GMM/
keys/DF/CM/LFCC-GMM/score.txt
keys/DF/CM/LFCC-LCNN/
keys/DF/CM/LFCC-LCNN/score.txt
keys/DF/CM/RawNet2/
keys/DF/CM/RawNet2/score.txt
keys/DF/CM/CQCC-GMM/
keys/DF/CM/CQCC-GMM/score.txt
keys/DF/README.txt
keys/LA/
keys/LA/README.txt
keys/LA/ASV/
keys/LA/ASV/ASVTorch_Kaldi/
keys/LA/ASV/ASVTorch_Kaldi/score.txt
keys/LA/ASV/trial_metadata.txt
keys/LA/CM/
keys/LA/CM/trial_metadata.txt
keys/LA/CM/LFCC-GMM/
keys/LA/CM/LFCC-GMM/score.txt
keys/LA/CM/LFCC-LCNN/
keys/LA/CM/LFCC-LCNN/score.txt
keys/LA/CM/RawNet2/
keys/LA/CM/RawNet2/score.txt
keys/LA/CM/CQCC-GMM/
keys/LA/CM/CQCC-GMM/score.txt
keys/LA/LA-C012-eval.npy
keys/LA/LA-C012-prog.npy
keys/LA/LA-C012-hidden.npy
keys/PA/
keys/PA/ASV/
keys/PA/ASV/trial_metadata.txt
keys/PA/ASV/ASVTorch_Kaldi/
keys/PA/ASV/ASVTorch_Kaldi/score.txt
keys/PA/CM/
keys/PA/CM/trial_metadata.txt
keys/PA/CM/LFCC-GMM/
keys/PA/CM/LFCC-GMM/score.txt
keys/PA/CM/LFCC-LCNN/
keys/PA/CM/LFCC-LCNN/score.txt
ke

In [25]:
!rm -r /content/drive/MyDrive/ASVspoof2021/keys/keys


In [19]:
!ls -lh /content/drive/MyDrive/ASVspoof2021/keys/


total 704M
drwx------ 3 root root 4.0K Oct  1  2022 DF
-rw------- 1 root root  25M Nov 28 13:36 DF-keys-full.tar.gz
-rw------- 1 root root  25M Nov 28 13:36 DF-keys-full.tar.gz.1
-rw------- 1 root root  25M Nov 28 13:36 DF-keys-full.tar.gz.2
-rw------- 1 root root  25M Nov 28 13:36 DF-keys-full.tar.gz.3
-rw------- 1 root root  25M Nov 28 13:36 DF-keys-full.tar.gz.4
-rw------- 1 root root  25M Nov 28 13:36 DF-keys-full.tar.gz.5
drwx------ 2 root root 4.0K Apr 13  2023 LA
-rw------- 1 root root  21M Nov 28 13:36 LA-keys-full.tar.gz
-rw------- 1 root root  21M Nov 28 13:36 LA-keys-full.tar.gz.1
-rw------- 1 root root  21M Nov 28 13:36 LA-keys-full.tar.gz.2
-rw------- 1 root root  21M Nov 28 13:36 LA-keys-full.tar.gz.3
-rw------- 1 root root  21M Nov 28 13:36 LA-keys-full.tar.gz.4
-rw------- 1 root root  21M Nov 28 13:36 LA-keys-full.tar.gz.5
drwx------ 2 root root 4.0K Apr  4 13:03 PA
-rw------- 1 root root  73M Nov 28 13:36 PA-keys-full.tar.gz
-rw------- 1 root root  73M Nov 28 13:36 PA-

In [10]:
meta_file = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt"

with open(meta_file, "r") as f:
    lines = f.readlines()

print(f"✅ Loaded {len(lines)} metadata entries.")



✅ Loaded 611829 metadata entries.


In [17]:
metadata_path = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt"

import pandas as pd

df_metadata = pd.read_csv(metadata_path, sep=" ", header=None)
df_metadata.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,LA_0023,DF_E_2000011,nocodec,asvspoof,A14,spoof,notrim,progress,traditional_vocoder,-,-,-,-
1,TEF2,DF_E_2000013,low_m4a,vcc2020,Task1-team20,spoof,notrim,eval,neural_vocoder_nonautoregressive,Task1,team20,FF,E
2,TGF1,DF_E_2000024,mp3m4a,vcc2020,Task2-team12,spoof,notrim,eval,traditional_vocoder,Task2,team12,FF,G
3,LA_0043,DF_E_2000026,mp3m4a,asvspoof,A09,spoof,notrim,eval,traditional_vocoder,-,-,-,-
4,LA_0021,DF_E_2000027,mp3m4a,asvspoof,A12,spoof,notrim,eval,neural_vocoder_autoregressive,-,-,-,-


In [13]:
import tarfile

key_archive_path = "/content/drive/MyDrive/ASVspoof2021/keys/DF-keys-full.tar.gz"
extract_dir = "/content/drive/MyDrive/ASVspoof2021/keys/DF"

with tarfile.open(key_archive_path, "r:gz") as tar:
    tar.extractall(path=extract_dir)

print("✅ Extracted DF metadata keys to:", extract_dir)


✅ Extracted DF metadata keys to: /content/drive/MyDrive/ASVspoof2021/keys/DF


In [16]:
!ls -lh /content/drive/MyDrive/ASVspoof2021


total 28M
drwx------ 5 root root 4.0K Apr  4 11:31 ASVspoof2021_DF_eval
-rw------- 1 root root  21M Apr  6 15:38 ASVspoof2021_DF_eval_part00.tar.gz
drwx------ 3 root root 4.0K Apr  6 12:55 ASVspoof2021_DF_eval_test_extract
drwx------ 2 root root 4.0K Apr  6 09:54 DF_download
drwx------ 2 root root 4.0K Apr  6 14:42 extracted_bonafide
drwx------ 5 root root 4.0K Apr  4 11:32 keys
-rw------- 1 root root 7.0M Apr  6 15:58 rawnet2_trained.pth


In [18]:
df_metadata.columns = [
    "Speaker_ID", "Utterance_ID", "Codec", "Dataset", "System_ID",
    "Label", "Trim", "Subset", "Attack_Type", "Extra1", "Extra2", "Extra3", "Extra4"
]

df_metadata.head()


,Speaker_ID,Utterance_ID,Codec,Dataset,System_ID,Label,Trim,Subset,Attack_Type,Extra1,Extra2,Extra3,Extra4
0,LA_0023,DF_E_2000011,nocodec,asvspoof,A14,spoof,notrim,progress,traditional_vocoder,-,-,-,-
1,TEF2,DF_E_2000013,low_m4a,vcc2020,Task1-team20,spoof,notrim,eval,neural_vocoder_nonautoregressive,Task1,team20,FF,E
2,TGF1,DF_E_2000024,mp3m4a,vcc2020,Task2-team12,spoof,notrim,eval,traditional_vocoder,Task2,team12,FF,G
3,LA_0043,DF_E_2000026,mp3m4a,asvspoof,A09,spoof,notrim,eval,traditional_vocoder,-,-,-,-
4,LA_0021,DF_E_2000027,mp3m4a,asvspoof,A12,spoof,notrim,eval,neural_vocoder_autoregressive,-,-,-,-


In [19]:
df_metadata["Label"].value_counts()


,count
Label,
spoof,589212
bonafide,22617


In [16]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch
import torch.nn as nn


count_spoof = (df_metadata['Label'] == 'spoof').sum()
count_bonafide = (df_metadata['Label'] == 'bonafide').sum()


class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=np.array([0] * count_spoof + [1] * count_bonafide))

class_weights = torch.tensor(class_weights, dtype=torch.float32)
print("Computed class weights:", class_weights)


criterion = nn.CrossEntropyLoss(weight=class_weights)



Computed class weights: tensor([ 0.5192, 13.5259])


In [20]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.utils.class_weight import compute_class_weight


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


count_spoof = 589212
count_bonafide = 22617


labels = np.array([0] * count_spoof + [1] * count_bonafide)
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        self.fc = nn.Linear(10, 2)

    def forward(self, x):
        return self.fc(x)

model = YourModel().to(device)


criterion = nn.CrossEntropyLoss(weight=class_weights)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Model, loss function, and optimizer set up successfully!")



Using device: cpu
Model, loss function, and optimizer set up successfully!


In [21]:
import torch
import torchaudio
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class AudioDeepfakeDataset(Dataset):
    def __init__(self, metadata_path, audio_dir, transform=None):
        """
        Args:
            metadata_path (str): Path to the metadata file (CSV or TXT).
            audio_dir (str): Directory where the audio files are stored.
            transform (callable, optional): Optional transform to apply on the data.
        """
        self.metadata = pd.read_csv(metadata_path, sep="\t")
        self.audio_dir = audio_dir
        self.transform = transform

        self.label_map = {'bonafide': 1, 'spoof': 0}

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        audio_path = f"{self.audio_dir}/{row['Utterance_ID']}.wav"
        label = self.label_map[row['Label']]

        waveform, sample_rate = torchaudio.load(audio_path)

        if self.transform:
            waveform = self.transform(waveform)

        return waveform, label


In [22]:
import torchaudio.transforms as T

transform = T.Resample(orig_freq=48000, new_freq=16000)


In [23]:
import pandas as pd

metadata_path = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt"


col_names = ["Speaker_ID", "Utterance_ID", "Codec", "Dataset", "System_ID", "Label",
             "Trim", "Subset", "Attack_Type", "Extra1", "Extra2", "Extra3", "Extra4"]


df = pd.read_csv(metadata_path, sep="\s+", names=col_names, header=None, engine="python")

print(df.head())
print(df.columns)

  Speaker_ID  Utterance_ID    Codec   Dataset     System_ID  Label    Trim  \
0    LA_0023  DF_E_2000011  nocodec  asvspoof           A14  spoof  notrim   
1       TEF2  DF_E_2000013  low_m4a   vcc2020  Task1-team20  spoof  notrim   
2       TGF1  DF_E_2000024   mp3m4a   vcc2020  Task2-team12  spoof  notrim   
3    LA_0043  DF_E_2000026   mp3m4a  asvspoof           A09  spoof  notrim   
4    LA_0021  DF_E_2000027   mp3m4a  asvspoof           A12  spoof  notrim   

     Subset                       Attack_Type Extra1  Extra2 Extra3 Extra4  
0  progress               traditional_vocoder      -       -      -      -  
1      eval  neural_vocoder_nonautoregressive  Task1  team20     FF      E  
2      eval               traditional_vocoder  Task2  team12     FF      G  
3      eval               traditional_vocoder      -       -      -      -  
4      eval     neural_vocoder_autoregressive      -       -      -      -  
Index(['Speaker_ID', 'Utterance_ID', 'Codec', 'Dataset', 'System_ID',

In [24]:
from torch.utils.data import Dataset

class AudioDataset(Dataset):
    def __init__(self, metadata_file, audio_dir, transform=None):
        self.metadata = pd.read_csv(metadata_file, sep="\s+", names=col_names, header=None, engine="python")
        self.audio_dir = audio_dir
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]

        audio_path = f"{self.audio_dir}/{row['Utterance_ID']}.wav"

        waveform, sample_rate = torchaudio.load(audio_path)

        label = 0 if row["Label"] == "bonafide" else 1  # bonafide = 0, spoof = 1


        if self.transform:
            waveform = self.transform(waveform)

        return waveform, label


In [22]:
%cd /content/drive/MyDrive/ASVspoof2021/



/content/drive/MyDrive/ASVspoof2021


In [25]:
import os

data_path = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"
print("Exists?", os.path.exists(data_path))
print("Sample files:", os.listdir(data_path)[:10])  # Show first 10 files


Exists? True
Sample files: ['DF_E_2753491.flac', 'DF_E_2000946.flac', 'DF_E_2751336.flac', 'DF_E_2753263.flac', 'DF_E_2001196.flac', 'DF_E_2003721.flac', 'DF_E_2752618.flac', 'DF_E_2751843.flac', 'DF_E_2750974.flac', 'DF_E_2751454.flac']


In [26]:
import os

folder_path = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"
files = os.listdir(folder_path)
print("Number of files:", len(files))
print("First 10 files:", files[:10])


Number of files: 2232
First 10 files: ['DF_E_2753491.flac', 'DF_E_2000946.flac', 'DF_E_2751336.flac', 'DF_E_2753263.flac', 'DF_E_2001196.flac', 'DF_E_2003721.flac', 'DF_E_2752618.flac', 'DF_E_2751843.flac', 'DF_E_2750974.flac', 'DF_E_2751454.flac']


In [27]:
import torchaudio

sample_file = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000011.flac"
waveform, sample_rate = torchaudio.load(sample_file)
print("✅ Loaded Successfully! Shape:", waveform.shape, "Sample Rate:", sample_rate)


✅ Loaded Successfully! Shape: torch.Size([1, 22801]) Sample Rate: 16000


In [28]:
input_dir = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"


In [29]:
import os

print("Exists?", os.path.exists(input_dir))
print("Sample files:", os.listdir(input_dir)[:10])  # Show first 10 files


Exists? True
Sample files: ['DF_E_2753491.flac', 'DF_E_2000946.flac', 'DF_E_2751336.flac', 'DF_E_2753263.flac', 'DF_E_2001196.flac', 'DF_E_2003721.flac', 'DF_E_2752618.flac', 'DF_E_2751843.flac', 'DF_E_2750974.flac', 'DF_E_2751454.flac']


In [30]:
import torchaudio

sample_file = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000011.flac"

waveform, sample_rate = torchaudio.load(sample_file)
print(f"✅ Waveform Shape: {waveform.shape}, Sample Rate: {sample_rate}")



✅ Waveform Shape: torch.Size([1, 22801]), Sample Rate: 16000


In [30]:
import os

meta_dir = "/content/drive/MyDrive/ASVspoof2021/keys/DF"
for root, dirs, files in os.walk(meta_dir):
    for file in files:
        print(file)


README.txt
trial_metadata.txt
score.txt
score.txt
score.txt
score.txt
README.txt
trial_metadata.txt
score.txt
score.txt
score.txt
score.txt


In [31]:
meta_file = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt"

with open(meta_file, "r") as f:
    lines = f.readlines()

print(f"Loaded {len(lines)} metadata entries.")


Loaded 611829 metadata entries.


In [32]:
for i in range(5):
    print(lines[i].strip())


LA_0023 DF_E_2000011 nocodec asvspoof A14 spoof notrim progress traditional_vocoder - - - -
TEF2 DF_E_2000013 low_m4a vcc2020 Task1-team20 spoof notrim eval neural_vocoder_nonautoregressive Task1 team20 FF E
TGF1 DF_E_2000024 mp3m4a vcc2020 Task2-team12 spoof notrim eval traditional_vocoder Task2 team12 FF G
LA_0043 DF_E_2000026 mp3m4a asvspoof A09 spoof notrim eval traditional_vocoder - - - -
LA_0021 DF_E_2000027 mp3m4a asvspoof A12 spoof notrim eval neural_vocoder_autoregressive - - - -


In [33]:
meta_info = []

for line in lines:
    parts = line.strip().split()
    if len(parts) >= 4:
        utt_id = parts[1]
        label = parts[-1]
        meta_info.append((utt_id, label))

print(f"Sample metadata entries: {meta_info[:5]}")


Sample metadata entries: [('DF_E_2000011', '-'), ('DF_E_2000013', 'E'), ('DF_E_2000024', 'G'), ('DF_E_2000026', '-'), ('DF_E_2000027', '-')]


In [34]:
import os
import torchaudio

input_dir = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"
output_dir = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/wav_for_rawnet"
os.makedirs(output_dir, exist_ok=True)

meta_file = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt"


with open(meta_file, "r") as f:
    lines = f.readlines()


file_ids = [line.strip().split()[0] for line in lines]


for file_id in file_ids:
    flac_path = os.path.join(input_dir, file_id + ".flac")
    wav_path = os.path.join(output_dir, file_id + ".wav")

    if os.path.exists(flac_path):
        waveform, sample_rate = torchaudio.load(flac_path)
        torchaudio.save(wav_path, waveform, sample_rate)

print("✅ Conversion complete!")


✅ Conversion complete!


In [35]:
import torch
import torchaudio
import numpy as np

def load_audio(filepath, sample_rate=16000, max_length=64000):
    waveform, sr = torchaudio.load(filepath)

    if sr != sample_rate:
        resampler = torchaudio.transforms.Resample(sr, sample_rate)
        waveform = resampler(waveform)

    # Mono
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    return waveform

def pre_emphasis(waveform, coeff=0.97):
    """Pre-emphasis filter"""
    waveform = waveform.squeeze(0)
    emphasized = torch.cat([waveform[:1], waveform[1:] - coeff * waveform[:-1]])
    return emphasized.unsqueeze(0)

def normalize_waveform(waveform):
    mean = waveform.mean()
    std = waveform.std()
    return (waveform - mean) / (std + 1e-5)

def pad_or_trim(waveform, max_length=64000):
    length = waveform.shape[1]
    if length < max_length:
        padding = max_length - length
        waveform = torch.nn.functional.pad(waveform, (0, padding))
    else:
        waveform = waveform[:, :max_length]
    return waveform

def extract_features(filepath):
    waveform = load_audio(filepath)
    waveform = pre_emphasis(waveform)
    waveform = normalize_waveform(waveform)
    waveform = pad_or_trim(waveform)
    return waveform


In [36]:
test_file = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/wav/DF_E_2000011.wav"



In [37]:
import os

base_dir = "/content/drive/MyDrive/ASVspoof2021"

for root, dirs, files in os.walk(base_dir):
    for name in dirs:
        print("📁 Folder:", os.path.join(root, name))
    for name in files:
        if name.endswith(".wav"):
            print("🎧 WAV File:", os.path.join(root, name))
            break



📁 Folder: /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval
📁 Folder: /content/drive/MyDrive/ASVspoof2021/keys
📁 Folder: /content/drive/MyDrive/ASVspoof2021/DF_download
📁 Folder: /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval_test_extract
📁 Folder: /content/drive/MyDrive/ASVspoof2021/extracted_bonafide
📁 Folder: /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac
📁 Folder: /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/wav_for_rawnet
📁 Folder: /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/balanced_dataset
📁 Folder: /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/ASVspoof2021_DF_eval
📁 Folder: /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/ASVspoof2021_DF_eval/flac
📁 Folder: /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/balanced_dataset/bonafide
📁 Folder: /content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/balanced_dataset/spoof
📁 Folder: /content/drive/MyDrive/ASVspoof2021/keys/DF
📁 Folder: /co

In [38]:
import os

folder = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/wav_for_rawnet"

print("Total files:", len(os.listdir(folder)))
print("Sample files:", os.listdir(folder)[:5])


Total files: 0
Sample files: []


In [41]:
import os

folder = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM"
print("Files in folder:", os.listdir(folder))


Files in folder: ['RawNet2', 'LFCC-GMM', 'LFCC-LCNN', 'CQCC-GMM', 'trial_metadata.txt']


In [42]:
import os

cm_dir = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM"
print("📁 Contents of CM directory:", os.listdir(cm_dir))


📁 Contents of CM directory: ['RawNet2', 'LFCC-GMM', 'LFCC-LCNN', 'CQCC-GMM', 'trial_metadata.txt']


In [43]:
import os


flac_dir = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"
meta_file = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt"


assert os.path.exists(flac_dir), "FLAC directory not found!"
assert os.path.exists(meta_file), "Metadata file not found!"


file_label_pairs = []
with open(meta_file, "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 4:
            continue

        file_name = parts[1]
        label = 1 if parts[3] == "bonafide" else 0
        full_path = os.path.join(flac_dir, file_name + ".flac")

        if os.path.exists(full_path):
            file_label_pairs.append((full_path, label))

print(f"✅ Parsed {len(file_label_pairs)} valid metadata entries.")
print("🔹 Sample:", file_label_pairs[:3])




✅ Parsed 2231 valid metadata entries.
🔹 Sample: [('/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000011.flac', 0), ('/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000013.flac', 0), ('/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000024.flac', 0)]


In [44]:
import torch
import torchaudio
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random


class RawAudioDataset(Dataset):
    def __init__(self, file_label_pairs, target_length=48000, transform=None):
        self.file_label_pairs = file_label_pairs
        self.target_length = target_length
        self.transform = transform

    def __len__(self):
        return len(self.file_label_pairs)

    def __getitem__(self, idx):
        audio_path, label = self.file_label_pairs[idx]

        try:
            waveform, sample_rate = torchaudio.load(audio_path)
        except Exception as e:
            print(f"⚠️ Failed to load {audio_path}: {e}")
            return torch.zeros(1, self.target_length), torch.tensor(0.0)


        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)


        if waveform.shape[1] < self.target_length:
            pad_len = self.target_length - waveform.shape[1]
            waveform = F.pad(waveform, (0, pad_len))
        else:
            waveform = waveform[:, :self.target_length]

        return waveform, torch.tensor(label, dtype=torch.float)


random.shuffle(file_label_pairs)
total_len = len(file_label_pairs)
train_len = int(0.8 * total_len)

train_files = file_label_pairs[:train_len]
val_files = file_label_pairs[train_len:]


train_dataset = RawAudioDataset(train_files)
val_dataset = RawAudioDataset(val_files)


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print(f"✅ Dataloaders ready! Train: {len(train_dataset)}, Val: {len(val_dataset)}")



✅ Dataloaders ready! Train: 1784, Val: 447


In [45]:
parsed_data = [
    ("/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/" + line.split()[1] + ".flac",
     0 if line.split()[5] == "bonafide" else 1)
    for line in open("/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt").readlines()
    if os.path.exists("/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/" + line.split()[1] + ".flac")
]


In [44]:
import os

audio_dir = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/"
flac_files = [f for f in os.listdir(audio_dir) if f.endswith(".flac")]
print(f"🎧 Total audio files found: {len(flac_files)}")


🎧 Total audio files found: 2231


In [46]:
from collections import Counter

metadata_path = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt"
audio_dir = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/"

parsed_data = []

with open(metadata_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 6:
            continue

        file_id = parts[1]
        label_str = parts[5]
        file_path = os.path.join(audio_dir, file_id + ".flac")

        if os.path.exists(file_path):
            label = 0 if label_str == "bonafide" else 1
            parsed_data.append((file_path, label))

# Count labels
label_counts = Counter([label for _, label in parsed_data])
print(f"🧾 Parsed data size: {len(parsed_data)}")
print(f"✅ Label distribution: {label_counts}")


🧾 Parsed data size: 2231
✅ Label distribution: Counter({1: 2146, 0: 85})


In [47]:
for i in range(5):
    print(parsed_data[i])


('/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000011.flac', 1)
('/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000013.flac', 1)
('/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000024.flac', 1)
('/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000026.flac', 1)
('/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000027.flac', 1)


In [48]:
import os
import tarfile


archive_path = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval.tar.gz"
extract_to = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"


In [49]:
import os
from collections import defaultdict

metadata_path = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt"
audio_base_path = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac/"

bonafide_list = []
spoof_list = []

with open(metadata_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        file_id = parts[1]
        label = parts[5]

        file_path = os.path.join(audio_base_path, f"{file_id}.flac")
        if os.path.exists(file_path):
            if label == "bonafide":
                bonafide_list.append((file_path, 0))
            else:
                spoof_list.append((file_path, 1))


In [50]:
import random

target_size = min(len(bonafide_list), len(spoof_list))
print(f"✅ Using {target_size} spoof and bonafide samples each.")

bonafide_sampled = random.sample(bonafide_list, target_size)
spoof_sampled = random.sample(spoof_list, target_size)

balanced_data = bonafide_sampled + spoof_sampled
random.shuffle(balanced_data)


from collections import Counter
labels = [label for _, label in balanced_data]
print("✅ Balanced label distribution:", Counter(labels))


✅ Using 85 spoof and bonafide samples each.
✅ Balanced label distribution: Counter({0: 85, 1: 85})


In [51]:
import os
import random

# Paths
flac_dir = "/content/drive/MyDrive/ASVspoof2021/ASVspoof2021_DF_eval/flac"


metadata_path = "/content/drive/MyDrive/ASVspoof2021/keys/DF/CM/trial_metadata.txt"
bonafide_ids = set()

with open(metadata_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        if parts[5].lower() == "bonafide":
            bonafide_ids.add(parts[1])


all_flacs = [f for f in os.listdir(flac_dir) if f.endswith(".flac")]
bonafide_flacs = [f for f in all_flacs if f[:-5] in bonafide_ids]
spoof_flacs = [f for f in all_flacs if f[:-5] not in bonafide_ids]


bonafide_selected = bonafide_flacs[:93]
spoof_selected = random.sample(spoof_flacs, 93)

print(f"🎯 Bonafide selected: {len(bonafide_selected)}")
print(f"🎯 Spoof selected: {len(spoof_selected)}")


🎯 Bonafide selected: 85
🎯 Spoof selected: 93


In [52]:
import random
import numpy as np


bonafide_samples = [s for s in parsed_data if s[1] == 0]
spoof_samples = [s for s in parsed_data if s[1] == 1]


target_per_class = min(len(bonafide_samples), len(spoof_samples), 1000)

bonafide_balanced = random.sample(bonafide_samples, target_per_class)
spoof_balanced = random.sample(spoof_samples, target_per_class)


balanced_data = bonafide_balanced + spoof_balanced
random.shuffle(balanced_data)


from sklearn.model_selection import train_test_split

X = [x[0] for x in balanced_data]
y = [x[1] for x in balanced_data]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

train_data = list(zip(X_train, y_train))
val_data = list(zip(X_val, y_val))

print("✅ Train label counts:", np.bincount(y_train))
print("✅ Val label counts:", np.bincount(y_val))


✅ Train label counts: [68 68]
✅ Val label counts: [17 17]


In [53]:
import random


bonafide_samples = [sample for sample in parsed_data if sample[1] == 0]
spoof_samples = [sample for sample in parsed_data if sample[1] == 1]


downsampled_spoof = random.sample(spoof_samples, len(bonafide_samples))


balanced_data = bonafide_samples + downsampled_spoof
random.shuffle(balanced_data)


import numpy as np
labels = [label for _, label in balanced_data]
print("✅ Balanced label distribution:", np.bincount(labels))


✅ Balanced label distribution: [85 85]


In [54]:
from sklearn.model_selection import train_test_split

X = [x[0] for x in balanced_data]
y = [x[1] for x in balanced_data]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

train_data = list(zip(X_train, y_train))
val_data = list(zip(X_val, y_val))

print("🧪 Train label counts:", np.bincount(y_train))
print("🧪 Val label counts:", np.bincount(y_val))



🧪 Train label counts: [68 68]
🧪 Val label counts: [17 17]


In [55]:
train_dataset = RawAudioDataset(train_data)
val_dataset = RawAudioDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)



In [56]:
val_labels_only = [label for _, label in val_data]
print("🧪 Val label counts:", np.bincount(val_labels_only))


🧪 Val label counts: [17 17]


In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RawNet2(nn.Module):
    def __init__(self):
        super(RawNet2, self).__init__()


        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=3, stride=3),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )


        self.resblock1 = self._make_resblock(128, 128)
        self.resblock2 = self._make_resblock(128, 256)
        self.resblock3 = self._make_resblock(256, 512)


        self.gru = nn.GRU(input_size=512, hidden_size=128, num_layers=1, batch_first=True)
        self.fc = nn.Linear(128, 1)

    def _make_resblock(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(out_channels),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.resblock1(x) + x
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = x.permute(0, 2, 1)  # [B, T, C]
        _, h_n = self.gru(x)
        h_n = h_n.squeeze(0)
        out = self.fc(h_n)
        return out


In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RawNet2().to(device)
criterion = torch.nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [59]:
batch_size = 4



In [60]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [61]:
import torch
torch.cuda.empty_cache()


In [62]:
import torch
import torch.nn as nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RawNet2().to(device)




for param in model.parameters():
    param.requires_grad = False


for param in model.fc.parameters():
    param.requires_grad = True


criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

def train(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for features, labels in dataloader:
            features, labels = features.to(device), labels.to(device).float()

            optimizer.zero_grad()
            outputs = model(features).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = torch.round(torch.sigmoid(outputs))
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}, Accuracy = {acc:.4f}")




In [68]:
train(model, train_loader, criterion, optimizer)



Epoch 1: Loss = 23.8598, Accuracy = 0.4485
Epoch 2: Loss = 23.5800, Accuracy = 0.5000
Epoch 3: Loss = 23.5141, Accuracy = 0.5074
Epoch 4: Loss = 23.4110, Accuracy = 0.5147
Epoch 5: Loss = 23.2845, Accuracy = 0.5294


In [69]:
torch.save(model.state_dict(), 'rawnet2_trained.pth')


In [70]:
torch.save(model.state_dict(), '/content/drive/MyDrive/rawnet2_trained.pth')


In [71]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, val_loader, threshold=0.5):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs).squeeze()

            preds = (outputs > threshold).int()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"✅ Accuracy: {accuracy:.4f}")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1-score: {f1:.4f}")


In [72]:
evaluate_model(model, val_loader)


✅ Accuracy: 0.5000
✅ Precision: 0.0000
✅ Recall: 0.0000
✅ F1-score: 0.0000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
